In [1]:
%%capture
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install detectron2 -f \https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.7/index.html

In [2]:
import gc
import os
from pathlib import Path
import random
import sys
import json

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import scipy as sp
import torch


import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, HTML

# --- plotly ---
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = "plotly_dark"

# --- models ---
from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

# --- setup ---
pd.set_option('display.max_columns', 50)

In [ ]:
# --- configs ---
thing_classes = [
    "bina",
    "yol_kesisimi",
    "futbol_sahasi",
    "silo"
]

In [ ]:
from detectron2.data import MetadataCatalog

MetadataCatalog.get("sayzek_train").set(thing_classes=thing_classes)


namespace(name='sayzek_train',
          thing_classes=['bina', 'yol_kesisimi', 'futbol_sahası', 'silo'])

In [ ]:
import json
import cv2
import os
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo

# Setting the trained model and configurations
def setup_cfg(model_weights, num_classes=4):
    cfg = get_cfg()
    config_name = "COCO-Detection/retinanet_R_50_FPN_3x.yaml"  
    cfg.merge_from_file(model_zoo.get_config_file(config_name))
    cfg.MODEL.WEIGHTS = model_weights  
    cfg.MODEL.RETINANET.NUM_CLASSES = num_classes  
    cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0  
    cfg.DATALOADER.NUM_WORKERS = 2  
    cfg.SOLVER.IMS_PER_BATCH = 2  
    return cfg

# Making predictions and saving as json format. 
def infer_and_save_results(cfg, test_images_path, image_file_name_to_image_id, output_json_path):
    predictor = DefaultPredictor(cfg)
    results = []

    for img_name in os.listdir(test_images_path):
        image_path = os.path.join(test_images_path, img_name)
        image = cv2.imread(image_path)

        outputs = predictor(image)
        instances = outputs["instances"].to("cpu")

        bboxes = instances.pred_boxes.tensor.numpy()
        labels = instances.pred_classes.numpy()
        scores = instances.scores.numpy()

        img_id = image_file_name_to_image_id[img_name]

        for bbox, label, score in zip(bboxes, labels, scores):

            bbox[2], bbox[3] = bbox[2] - bbox[0], bbox[3] - bbox[1]  

            res = {
                'image_id': img_id,
                'category_id': int(label) + 1, 
                'bbox': list(bbox.astype('float64')),
                'score': float("{:.8f}".format(score))
            }
            results.append(res)

    with open(output_json_path, 'w') as f:
        json.dump(results, f)

if __name__ == "__main__":
    model_weights = "/kaggle/input/final-retinanet-model/model_final.pth"  
    test_images_path = '/kaggle/input/test-data/test-images'  
    image_file_name_to_image_id = json.load(open('/kaggle/input/test-annotations/test_image_file_name_to_image_id.json')) 
    output_json_path = 'KadircanIdrisoglu.json'  

    cfg = setup_cfg(model_weights)

    thing_classes = ["bina", "yol_kesisimi", "futbol_sahasi", "silo"]
    MetadataCatalog.get("sayzek_train").set(thing_classes=thing_classes)



    infer_and_save_results(cfg, test_images_path, image_file_name_to_image_id, output_json_path)


/opt/conda/lib/python3.10/site-packages/fvcore/common/checkpoint.py:252: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

/opt/conda/lib/python3.10/site-packages/torch/function